# Transfer Learning and Fine-Tuning with TensorFlow

# Step 1: Import libraries


In [8]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.19.0


# Step 2: Download and prepare the dataset

In [9]:
import os
import zipfile
import pathlib

# Check if dataset directory already exists

In [10]:
data_dir = pathlib.Path('cats_and_dogs_filtered')
if not data_dir.exists():
    print("Downloading cats and dogs dataset...")

# Download the dataset

In [11]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=False)

 # Create the directory

In [12]:
os.makedirs('cats_and_dogs_filtered', exist_ok=True)


# Manually extract the zip file

In [13]:
print("Extracting zip file...")
with zipfile.ZipFile(path_to_zip, 'r') as zip_ref:
    zip_ref.extractall('.')
    print("Dataset extraction complete!")
    

Extracting zip file...


Dataset extraction complete!


# Set path to the dataset

In [14]:
PATH = 'cats_and_dogs_filtered'

# Verify directories exist

In [15]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

# Print directory information to help debug

In [16]:
print(f"Train directory: {train_dir}")
print(f"Train directory exists: {os.path.exists(train_dir)}")
print(f"Validation directory: {validation_dir}")
print(f"Validation directory exists: {os.path.exists(validation_dir)}")


Train directory: cats_and_dogs_filtered\train
Train directory exists: True
Validation directory: cats_and_dogs_filtered\validation
Validation directory exists: True


# List subdirectories to confirm dataset structure

In [17]:
if os.path.exists(train_dir):
    print("Train subdirectories:", os.listdir(train_dir))
if os.path.exists(validation_dir):
    print("Validation subdirectories:", os.listdir(validation_dir))

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

Train subdirectories: ['cats', 'dogs']
Validation subdirectories: ['cats', 'dogs']


# Create train and validation datasets


In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE)


# TensorBoard setup for visualization
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, 
    histogram_freq=1
)

# Train the model
initial_epochs = 10
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=initial_epochs,
    callbacks=[cp_callback, tensorboard_callback]
)

# Step 10: Learning curves
# ----------------------
# Plot the training and validation accuracy/loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(12, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

# Step 11: Fine tuning
# ------------------
# Un-freeze the base model for fine-tuning
base_model.trainable = True

# Freeze all the layers before the `fine_tune_at` layer
fine_tune_at = 100  # Choose a layer from which to fine-tune
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Compile the model with a lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Much lower learning rate
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# View the trainable status of each layer
print("Number of layers in the base model:", len(base_model.layers))
for i, layer in enumerate(base_model.layers):
    print(i, layer.name, layer.trainable)

# Step 12: Continue training with fine-tuning
# ----------------------------------------
fine_tune_epochs = 10
total_epochs = initial_epochs + fine_tune_epochs

# Start from the epoch we left off
history_fine = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    callbacks=[cp_callback, tensorboard_callback]
)

# Step 13: Learning curves with fine-tuning
# --------------------------------------
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']
loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

plt.figure(figsize=(12, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.axvline(x=initial_epochs-1, color='r', linestyle='--')
plt.annotate('Start Fine Tuning', xy=(initial_epochs, 0.8), xytext=(initial_epochs, 0.65),
             arrowprops=dict(facecolor='black', shrink=0.05))
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.axvline(x=initial_epochs-1, color='r', linestyle='--')
plt.annotate('Start Fine Tuning', xy=(initial_epochs, 0.8), xytext=(initial_epochs, 0.25),
             arrowprops=dict(facecolor='black', shrink=0.05))
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

# Step 14: Evaluation and prediction
# -------------------------------
# Evaluate the model
loss, accuracy = model.evaluate(validation_dataset)
print('Test accuracy :', accuracy)

# Make predictions
for image_batch, labels_batch in validation_dataset.take(1):
    images = image_batch.numpy()
    labels = labels_batch.numpy()
    predictions = model.predict(image_batch)
    
    plt.figure(figsize=(10, 10))
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].astype('uint8'))
        
        prediction = np.argmax(predictions[i])
        if prediction == labels[i]:
            color = 'green'
        else:
            color = 'red'
        
        plt.title(f"Actual: {class_names[labels[i]]}\nPredicted: {class_names[prediction]}", color=color)
        plt.axis('off')
    plt.show()

# Step 15: Save the model
# ---------------------
model.save('cats_dogs_transfer_learning_model')
print("Model saved successfully")

# Step 16: Function to predict new images
# ------------------------------------
def predict_image(image_path):
    """
    Make a prediction on a single image
    """
    img = tf.keras.utils.load_img(
        image_path, 
        target_size=IMG_SIZE
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch dimension
    
    # Preprocess the image similarly to the training data
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    
    # Make prediction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions[0])
    confidence = np.max(predictions[0])
    
    # Display the image and prediction
    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.title(f"Prediction: {class_names[predicted_class]}\nConfidence: {confidence:.2f}")
    plt.axis('off')
    plt.show()
    
    return class_names[predicted_class], confidence

# Example usage
# predict_image('path_to_your_image.jpg')

# Class names

In [ ]:
class_names = train_dataset.class_names
print("Classes:", class_names)

# Step 3: Configure the dataset for performance

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

# Step 4: Data augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomZoom(0.2),
])

# Visualize a few augmented examples

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.title(class_names[labels[0]])
        plt.axis("off")

# Step 5: Rescale pixel values
# --------------------------
# Create a rescale layer for pixel normalization

In [ ]:
rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

# Step 6: Load pre-trained model
# ----------------------------
# Load MobileNetV2 without the classification head

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,),  # Input shape should include color channels
    include_top=False,  # Exclude the classification head
    weights='imagenet'  # Use pre-trained weights from ImageNet
)

# Freeze the base model to prevent its weights from being updated during training

In [ ]:
base_model.trainable = False

# Let's look at the base model architecture

In [ ]:
base_model.summary()

# Step 7: Create the feature extractor
# ---------------------------------
# Add a global average pooling layer and a dense output layer

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(len(class_names), activation='softmax')

# Combine all the layers into a model

In [ ]:
inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
x = data_augmentation(inputs)  # Apply data augmentation
x = rescale(x)  # Rescale pixel values
x = base_model(x, training=False)  # Pass the image through the base model
x = global_average_layer(x)  # Apply global average pooling
x = tf.keras.layers.Dropout(0.2)(x)  # Add dropout to prevent overfitting
outputs = prediction_layer(x)  # Add the dense output layer

model = tf.keras.Model(inputs, outputs)

# Step 8: Compile the model
# -----------------------

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)


# Display the model summary

In [ ]:
model.summary()

# Step 9: Train the model
# ---------------------
# Define callbacks

In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


# Create checkpoint callback

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
)